# Regresion lineal 2do ejercicio

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_percentage_error, root_mean_squared_error
import pickle
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoCV, RidgeCV

### Importación de datos

In [2]:
demografic_data = pd.read_csv("../data/raw/demographic_health_data.csv")
demografic_data

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,56037,43051,6104,14.178532,6326,14.694200,5359,12.448027,6577,15.277229,...,2098,8.9,8.3,9.6,2834,2.6,2.4,2.8,821,5
3136,56039,23081,2384,10.328842,2185,9.466661,2967,12.854729,4093,17.733200,...,928,7.2,6.5,8.0,1360,2.4,2.2,2.6,447,5
3137,56041,20299,3121,15.375142,3205,15.788955,2153,10.606434,2702,13.311001,...,1163,10.4,9.5,11.2,1500,3.0,2.8,3.2,430,5
3138,56043,7885,858,10.881420,1113,14.115409,715,9.067850,903,11.452124,...,506,11.3,10.3,12.1,686,3.4,3.2,3.7,207,6


In [ ]:
columnas = demografic_data.columns.to_list()
columnas

#### Observación de significado de datos

In [ ]:
dict_data_significados =  pd.read_csv("../data/raw/demographic_health_data_dict.csv", encoding="latin1")
dict_data_significados

### Primeras filas del dataset

In [ ]:
demografic_data.head()

### Últimas filas del dataset

In [ ]:
demografic_data.tail()

### Evaluación de datos y conteo de nulos

In [ ]:
demografic_data.info()

In [ ]:
demografic_data.isnull().sum()

### Estadisticas descriptivas de variables numércas

In [ ]:
demografic_data.describe().T.iloc[0:50] 


In [ ]:
demografic_data.describe().T.iloc[51:108]

#### Determinación de la no igualdad entre TOT_POP y Total Population. 

In [5]:
demografic_data['TOT_POP'] - demografic_data['Total Population']

0         565
1       14662
2       -1320
3        -180
4         173
        ...  
3135    -1476
3136      158
3137     -459
3138     -368
3139     -150
Length: 3140, dtype: int64

### Estadisticas descriptivas de variables categóricas

In [ ]:
demografic_data.describe(include='object').T

### Distribución de variables categóricas

In [ ]:
for col in demografic_data.select_dtypes(include='object').columns:
    print(demografic_data[col].value_counts())
    print()

In [ ]:
for col in demografic_data.columns:
    if col.find('diabetes') != -1:
        print(col)    

diabetes_prevalence

diabetes_Lower 95% CI

diabetes_Upper 95% CI

diabetes_number



### Elegimos diabetes_prevalence como variable objetivo

> Es una métrica clara de salud pública (la prevalencia de diabetes por condado).
> Tiene una relación previsible con muchas variables explicativas: obesidad, edad, ingresos, educación, acceso a salud.

Por lo tanto debemos eliminar todas las variables que puedan inducir al modelo a conocer la diabetes

### Limpieza de datos

In [10]:
demografic_data_clean = demografic_data.copy()

#### Motivo de eliminacion de columnas

- STATE_NAME/ COUNTY_NAME se eliminan por ser objetos
- Todos los relacionados con diabetes son eliminados por que son parte del target ('diabetes_Lower 95% CI', 'diabetes_Upper 95% CI', 'diabetes_number')
- 

In [21]:
cols_to_drop = ['STATE_NAME', 'COUNTY_NAME', 'diabetes_Lower 95% CI', 'diabetes_Upper 95% CI', 'diabetes_number']
demografic_data_clean.drop(columns=cols_to_drop)

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_prevalence,COPD_Lower 95% CI,COPD_Upper 95% CI,COPD_number,diabetes_prevalence,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,8.6,7.3,9.9,3644,12.9,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,8.6,7.2,10.1,14692,12.0,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,12.1,10.7,13.3,2373,19.7,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,10.0,8.8,11.3,1789,14.1,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,10.5,9.1,12.0,4661,13.5,3.4,3.2,3.7,1507,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,56037,43051,6104,14.178532,6326,14.694200,5359,12.448027,6577,15.277229,...,6.6,5.7,7.7,2098,8.9,2.6,2.4,2.8,821,5
3136,56039,23081,2384,10.328842,2185,9.466661,2967,12.854729,4093,17.733200,...,4.9,4.0,6.0,928,7.2,2.4,2.2,2.6,447,5
3137,56041,20299,3121,15.375142,3205,15.788955,2153,10.606434,2702,13.311001,...,8.0,7.0,9.2,1163,10.4,3.0,2.8,3.2,430,5
3138,56043,7885,858,10.881420,1113,14.115409,715,9.067850,903,11.452124,...,8.3,7.1,9.6,506,11.3,3.4,3.2,3.7,207,6


In [22]:
demografic_data_clean['diabetes_prevalence'].notnull().sum()

np.int64(3140)

#### Separar variables predictoras y objetivo

In [23]:
X = demografic_data_clean.drop(columns=['diabetes_prevalence'])
y = demografic_data_clean['diabetes_prevalence']

#### Separar conjunto de entrenamiento y prueba

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Train model

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
y_pred_lr.score

ValueError: could not convert string to float: 'Oceana'

### Lasso train

In [ ]:
lasso = Lasso(alpha=1.0)
lasso.fit(X_train, y_train)

y_pred_lasso = lasso.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

print(f"R² Lasso (α=1.0): {r2_lasso:.4f}")
print(f"RMSE Lasso: {rmse_lasso:.4f}")

### Ridge Train

In [ ]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

print(f"R² Ridge (α=1.0): {r2_ridge:.4f}")
print(f"RMSE Ridge: {rmse_ridge:.4f}")

### Comparar R

In [ ]:
alphas = np.linspace(0.0, 20.0, 50)

r2_lasso_list = []
r2_ridge_list = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    ridge = Ridge(alpha=alpha)
    
    lasso.fit(X_train, y_train)
    ridge.fit(X_train, y_train)
    
    r2_lasso_list.append(r2_score(y_test, lasso.predict(X_test)))
    r2_ridge_list.append(r2_score(y_test, ridge.predict(X_test)))

## Visualización del comparativo

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(alphas, r2_lasso_list, label='Lasso R²', marker='o')
plt.plot(alphas, r2_ridge_list, label='Ridge R²', marker='x')
plt.axhline(y=r2_lr, color='gray', linestyle='--', label='Lineal Base R²')
plt.xlabel('α (alpha)')
plt.ylabel('R² Score')
plt.title('Comparación de R² vs alpha (Lasso y Ridge)')
plt.legend()
plt.grid(True)
plt.show()

### Validación cruzada entre los 2 metodos

In [ ]:
lasso_cv = LassoCV(alphas=np.linspace(0.1, 20, 100), cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)

print(f"Mejor alpha para Lasso: {lasso_cv.alpha_:.4f}")

# Puedes evaluar el modelo óptimo así:
y_pred_lasso_cv = lasso_cv.predict(X_test)
r2_lasso_cv = r2_score(y_test, y_pred_lasso_cv)
rmse_lasso_cv = np.sqrt(mean_squared_error(y_test, y_pred_lasso_cv))

print(f"R² LassoCV: {r2_lasso_cv:.4f}")
print(f"RMSE LassoCV: {rmse_lasso_cv:.4f}")